In [ ]:
import sys
sys.path.append('/class')
from initspark import *
sc, spark, conf = initspark()


In [ ]:
docker network create network1
docker run --name cassandra1 --network network1 -d cassandra
docker run -it --network network1 --rm cassandra cqlsh cassandra1
docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' cassandra1


In [2]:
import os
CASSANDRA_IP=os.getenv('CASSANDRA1')
if CASSANDRA_IP is None:
    CASSANDRA_IP = '172.18.0.2'
print(CASSANDRA_IP)

from cassandra.cluster import Cluster
cluster = Cluster([CASSANDRA_IP])
session = cluster.connect()
session.execute('DROP KEYSPACE IF EXISTS classroom')
session.execute("CREATE KEYSPACE classroom WITH REPLICATION={'class':'SimpleStrategy', 'replication_factor':'1'}")
session = cluster.connect('classroom')
session.execute("create table student(id int PRIMARY KEY, firstname text, lastname text, emails set<text>)")
session.execute("insert into student (id, firstname, lastname, emails) values (1, 'Joe', 'Smith', {'joes@xyz.com', 'joe.smith@abc.net'})")
session.execute("update student set firstname = 'Joseph' where id = 1")
session.execute("insert into student (id, firstname, lastname, emails) values (2, 'Mike', 'Jones', {'mikej@xyz.com', 'mike.jones@def.net', 'mike1234@gmail.com'})")
rows = session.execute('SELECT id, firstname, lastname, emails from student')
print(list(rows))


172.18.0.2
[Row(id=1, firstname='Joseph', lastname='Smith', emails=SortedSet(['joe.smith@abc.net', 'joes@xyz.com'])), Row(id=2, firstname='Mike', lastname='Jones', emails=SortedSet(['mike.jones@def.net', 'mike1234@gmail.com', 'mikej@xyz.com']))]


In [3]:
import sys
sys.path.append('/class')
from initspark import *
sc, spark, conf = initspark(cassandra=CASSANDRA_IP)



initializing pyspark
pyspark initialized


In [4]:
people = spark.read.format("org.apache.spark.sql.cassandra").options(table="student", keyspace="classroom").load()
display(people)
print(people.collect())


,id,emails,firstname,lastname
0,1,"[joe.smith@abc.net, joes@xyz.com]",Joseph,Smith
1,2,"[mike.jones@def.net, mike1234@gmail.com, mikej...",Mike,Jones


[Row(id=1, emails=['joe.smith@abc.net', 'joes@xyz.com'], firstname='Joseph', lastname='Smith'), Row(id=2, emails=['mike.jones@def.net', 'mike1234@gmail.com', 'mikej@xyz.com'], firstname='Mike', lastname='Jones')]


In [9]:
# Append the results of a DataFrame into a Cassandra table
x = sc.parallelize([(3, 'Mary', 'Johnson', ['Mary1@gmail.com', 'Mary2@yahoo.com'])])
x1 = spark.createDataFrame(x, schema = ['id', 'firstname', 'lastname', 'emails'])
x1.write.format("org.apache.spark.sql.cassandra").options(table="student", keyspace="classroom").mode("append").save()



Py4JJavaError: An error occurred while calling o217.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 23, localhost, executor driver): java.lang.NoClassDefFoundError: com/twitter/jsr166e/LongAdder
	at org.apache.spark.metrics.OutputMetricsUpdater$TaskMetricsSupport$class.$init$(OutputMetricsUpdater.scala:107)
	at org.apache.spark.metrics.OutputMetricsUpdater$TaskMetricsUpdater.<init>(OutputMetricsUpdater.scala:153)
	at org.apache.spark.metrics.OutputMetricsUpdater$.apply(OutputMetricsUpdater.scala:75)
	at com.datastax.spark.connector.writer.TableWriter.writeInternal(TableWriter.scala:209)
	at com.datastax.spark.connector.writer.TableWriter.insert(TableWriter.scala:197)
	at com.datastax.spark.connector.writer.TableWriter.write(TableWriter.scala:183)
	at com.datastax.spark.connector.RDDFunctions$$anonfun$saveToCassandra$1.apply(RDDFunctions.scala:36)
	at com.datastax.spark.

In [8]:
people = spark.read.format("org.apache.spark.sql.cassandra").options(table="student", keyspace="classroom").load()
display(people)



,id,emails,firstname,lastname
0,1,"[joe.smith@abc.net, joes@xyz.com]",Joseph,Smith
1,2,"[mike.jones@def.net, mike1234@gmail.com, mikej...",Mike,Jones


In [6]:
people.createOrReplaceTempView('people')
people2 = spark.sql('select id, firstname, lastname, email from people LATERAL VIEW EXPLODE(emails) EXPLODED_TABLE AS email')
display(people2)



,id,firstname,lastname,email
0,1,Joseph,Smith,joe.smith@abc.net
1,1,Joseph,Smith,joes@xyz.com
2,2,Mike,Jones,mike.jones@def.net
3,2,Mike,Jones,mike1234@gmail.com
4,2,Mike,Jones,mikej@xyz.com


In [7]:
people3 = people2.where("email like '%.com'").orderBy("id")
display(people3)

,id,firstname,lastname,email
0,1,Joseph,Smith,joes@xyz.com
1,2,Mike,Jones,mike1234@gmail.com
2,2,Mike,Jones,mikej@xyz.com


In [10]:
import pymongo
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
classroom = client["classroom"]
if 'classroom' in (x['name'] for x in client.list_databases()):
    client.drop_database('classroom')

people = classroom['people']
name = {"firstname" : "Adam", "personid":4}
x = people.insert_one(name)

names = [{"firstname" : "Betty", "personid":5}
         ,{"firstname" : "Charlie", "personid":6}]
x = people.insert_many(names)

x = people.find()
print ('*' * 80)
print ('from mongo directly')
print (list(x))
print ('*' * 80)



********************************************************************************
from mongo directly
[{'_id': ObjectId('5e1de8c400921aaf2bc4456d'), 'firstname': 'Adam', 'personid': 4}, {'_id': ObjectId('5e1de8c400921aaf2bc4456e'), 'firstname': 'Betty', 'personid': 5}, {'_id': ObjectId('5e1de8c400921aaf2bc4456f'), 'firstname': 'Charlie', 'personid': 6}]
********************************************************************************


In [11]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext()
spark = SparkSession.builder.appName("myApp")\
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/classroom") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/classroom") \
    .getOrCreate()


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Test, master=local) created by __init__ at /class/initspark.py:20 

In [12]:
df = spark.read.format("mongo").option("uri", "mongodb://127.0.0.1/classroom.people").load()
df.show()


+--------------------+---------+--------+
|                 _id|firstname|personid|
+--------------------+---------+--------+
|[5e1de8c400921aaf...|     Adam|       4|
|[5e1de8c400921aaf...|    Betty|       5|
|[5e1de8c400921aaf...|  Charlie|       6|
+--------------------+---------+--------+



In [13]:
x = sc.parallelize([(7, 'David')])
x1 = spark.createDataFrame(x, schema = ['personid', 'firstname'])
x1.write.format("mongo").options(collection="people", database="classroom").mode("append").save()

In [14]:
df = spark.read.format("mongo").option("uri", "mongodb://127.0.0.1/classroom.people").load()
df.show()


+--------------------+---------+--------+
|                 _id|firstname|personid|
+--------------------+---------+--------+
|[5e1de8c400921aaf...|     Adam|       4|
|[5e1de8c400921aaf...|    Betty|       5|
|[5e1de8c400921aaf...|  Charlie|       6|
|[5e1de8da693e19cb...|    David|       7|
+--------------------+---------+--------+



In [15]:
df.createOrReplaceTempView('people')
spark.sql('select * from people').show()


+--------------------+---------+--------+
|                 _id|firstname|personid|
+--------------------+---------+--------+
|[5e1de8c400921aaf...|     Adam|       4|
|[5e1de8c400921aaf...|    Betty|       5|
|[5e1de8c400921aaf...|  Charlie|       6|
|[5e1de8da693e19cb...|    David|       7|
+--------------------+---------+--------+

